# Table Tutorial

## Part I: Preprocessing

In [1]:
# If necessary:
import os
os.remove('snorkel.db')

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from snorkel import SnorkelSession
session = SnorkelSession()

## Parse the `Corpus`

In [3]:
from snorkel.parser import CorpusParser
from snorkel.parser import HTMLParser
from snorkel.parser import OmniParser

doc_parser = HTMLParser(path='data/hardware/hardware100_html/')
context_parser = OmniParser()
cp = CorpusParser(doc_parser, context_parser, max_docs=100) 
# NOTE: max_docs may be being used!

In [4]:
%time corpus = cp.parse_corpus(name='Hardware', session=session)

[========================================] 100%

CPU times: user 4min 5s, sys: 9.69 s, total: 4min 14s
Wall time: 9min 31s


In [5]:
for doc in corpus.documents[:5]: print doc

Document LTSCS02912-1
Document DISES00192-1
Document PHGLS20125-1
Document bc550
Document 2N4123-D


### Saving the `Corpus`
Finally, we persist the parsed corpus in Snorkel's database backend:

In [6]:
session.add(corpus)
session.commit()

### Reloading the `Corpus`
If the corpus has already been parsed, load it here:

In [7]:
from snorkel.models import Corpus

corpus = session.query(Corpus).filter(Corpus.name == 'Hardware').one()
print "%s contains %d Documents" % (corpus, len(corpus))

Corpus (Hardware) contains 100 Documents


### Split the `Corpus` into Train/Dev/Test

In [8]:
# If necessary:
# train = session.query(Corpus).filter(Corpus.name == 'Hardware Training').one()
# session.delete(train)

# dev = session.query(Corpus).filter(Corpus.name == 'Hardware Development').one()
# session.delete(dev)

# test = session.query(Corpus).filter(Corpus.name == 'Hardware Test').one()
# session.delete(test)

# session.commit()

In [9]:
# from snorkel.utils import split_corpus

# split_corpus(session, corpus, train=0.75, development=0.25, test=0, seed=1)

In [10]:
# If necessary
import os
os.system('cp snorkel.db snorkel.db\ corpus');

Next, in Part 2, we will look at how to extract `Candidate` relations from our saved `Corpus`.